<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/otto_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install polars
!pip install 'polars[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 MB 33.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import polars as pl
paths = ['/content/drive/My Drive/OTTO/sub_pip_co-visitiation_577.csv', # 0.577
         '/content/drive/My Drive/OTTO/sub_pipeline2_577.csv', # 0.577
         '/content/drive/My Drive/OTTO/sub_rerank_577.csv' # 0.577
        ]

In [4]:
def read_sub(path, weight=1): # by default let us assing the weight of 1 to predictions from each submission, this will be akin to a standard vote ensemble
    '''a helper function for loading and preprocessing submissions'''
    return (
        pl.read_csv(path)
            .with_columns(pl.col('labels').str.split(by=' '))
            .with_columns(pl.lit(weight).alias('vote'))
            .explode('labels')
            .rename({'labels': 'aid'})
            .with_columns(pl.col('aid').cast(pl.UInt32)) # we are casting the `aids` to `Int32`! memory management is super important to ensure we don't run out of resources
            .with_columns(pl.col('vote').cast(pl.UInt8))
    )

In [ ]:
import polars as pl
weight=1
# sub1=pl.read_csv('/content/drive/My Drive/OTTO/sub_pip_co-visitiation_577.csv').with_columns(pl.col('labels').str.split(by=' ')).with_columns(pl.lit(weight).alias('vote'))
# sub2=pl.read_csv('/content/drive/My Drive/OTTO/sub_pipeline2_577.csv').with_columns(pl.col('labels').str.split(by=' ')).with_columns(pl.lit(weight).alias('vote'))
# sub3=pl.read_csv('/content/drive/My Drive/OTTO/sub_rerank_577.csv').with_columns(pl.col('labels').str.split(by=' ')).with_columns(pl.lit(weight).alias('vote'))

In [ ]:
# sub1=sub1.explode('labels').rename({'labels': 'aid'})
# sub2=sub2.explode('labels').rename({'labels': 'aid'})
# sub3=sub3.explode('labels').rename({'labels': 'aid'})

In [ ]:
# sub1=sub1.with_columns(pl.col('aid').cast(pl.UInt32)).with_columns(pl.col('vote').cast(pl.UInt8))
# sub2=sub2.with_columns(pl.col('aid').cast(pl.UInt32)).with_columns(pl.col('vote').cast(pl.UInt8))
# sub3=sub3.with_columns(pl.col('aid').cast(pl.UInt32)).with_columns(pl.col('vote').cast(pl.UInt8))

In [ ]:
# sub1.write_csv('/content/drive/My Drive/OTTO/sub1.csv')

In [ ]:
# import polars as pl
# sub1=pl.read_csv('/content/drive/My Drive/OTTO/sub1.csv')
# sub2=pl.read_csv('/content/drive/My Drive/OTTO/sub2.csv')
# sub3=pl.read_csv('/content/drive/My Drive/OTTO/sub3.csv')

In [6]:
subs = [read_sub(path) for path in paths]

In [7]:
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how='outer', on=['session_type', 'aid'], suffix='_right2')
subs.head()
# subs = sub1.join(sub2, how='outer', on=['session_type', 'aid'])

session_type,aid,vote,vote_right,vote_right2
str,u32,u8,u8,u8
"""12899779_click...",59625,1,1,1
"""12899779_click...",1253524,1,1,1
"""12899779_click...",737445,1,1,1
"""12899779_click...",438191,1,1,1
"""12899779_click...",731692,1,1,1


In [8]:
subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)

subs.head()

<ipython-input-8-a028f947c706>:1: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0
  subs = (subs


session_type,aid,vote_sum
str,u32,u8
"""14571581_carts...",196038,3
"""14571581_carts...",1780093,3
"""14571581_carts...",1497245,3
"""14571581_carts...",174670,3
"""14571581_carts...",1764910,3


In [10]:
%%time
# preds = subs.groupby('session_type').agg([
#     pl.col('aid').head(20).alias('labels')
# ]) 
preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))
# preds

<timed exec>:4: DeprecationWarning: `with_column` has been deprecated in favor of `with_columns`. This method will be removed in version 0.17.0


CPU times: user 3min 1s, sys: 4.43 s, total: 3min 6s
Wall time: 3min 5s


In [11]:
preds.write_csv('submission.csv')

In [12]:
preds.write_csv('/content/drive/My Drive/OTTO/submission_ens.csv')